In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from keras.layers import Activation, Dense, GRU, LSTM
from keras.optimizers import Adam, RMSprop

Using TensorFlow backend.


In [3]:
INPUT_FILE = "7078299.txt"
import re

# extract the input as a stream of characters
print("Extracting text from input...")
fin = open(INPUT_FILE, 'r', encoding='utf-8')
lines = []
for line in fin:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()

text = " ".join(lines)

Extracting text from input...


In [4]:
chars = text.split()
print (chars)

['\ufeffсвои', 'люди', '–', 'сочтемся', 'александр', 'николаевич', 'островский', '«липочка(сидит', 'у', 'окна', 'с', 'книгой).', 'какое', 'приятное', 'занятие', 'эти', 'танцы!', 'ведь', 'уж', 'как', 'хорошо!', 'что', 'может', 'быть', 'восхитительнее?', 'приедешь', 'в', 'собрание', 'али', 'к', 'кому', 'на', 'свадьбу,', 'сидишь,', 'натурально', '–', 'вся', 'в', 'цветах,', 'разодета,', 'как', 'игрушка', 'али', 'картинка', 'журнальная;', 'вдруг', 'подлетает', 'кавалер:', '«удостойте', 'счастия,', 'сударыня!»', 'ну,', 'видишь:', 'если', 'человек', 'с', 'понятием', 'али', 'армейской', 'какой', '–', 'возьмешь', 'да', 'и', 'прищуришься,', 'отвечаешь:', '«извольте,', 'с', 'удовольствием!»', 'ах!', '(с', 'жаром)', 'оча-ро-ва-тельно!', 'это', 'просто', 'уму', 'непостижимо!..»', 'александр', 'николаевич', 'островский', 'свои', 'люди', '–', 'сочтемся!', 'действующие', 'лица', 'самсон', 'силыч', 'большов,', 'купец.', 'аграфена', 'кондратьевна,', 'его', 'жена.', 'олимпиада', 'самсоновна', '(липочка),

In [5]:
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# creating lookup tables
# Here chars is the number of features in our character "vocabulary"
char2index

{'\ufeffсвои': 0,
 'люди': 3935,
 '–': 3858,
 'сочтемся': 3,
 'александр': 77,
 'николаевич': 78,
 'островский': 79,
 '«липочка(сидит': 7,
 'у': 4188,
 'окна': 124,
 'с': 4258,
 'книгой).': 126,
 'какое': 3651,
 'приятное': 128,
 'занятие': 129,
 'эти': 3896,
 'танцы!': 131,
 'ведь': 4078,
 'уж': 4141,
 'как': 4175,
 'хорошо!': 135,
 'что': 4219,
 'может': 3670,
 'быть': 1936,
 'восхитительнее?': 139,
 'приедешь': 140,
 'в': 4261,
 'собрание': 142,
 'али': 3194,
 'к': 4112,
 'кому': 145,
 'на': 4243,
 'свадьбу,': 147,
 'сидишь,': 148,
 'натурально': 149,
 'вся': 3914,
 'цветах,': 153,
 'разодета,': 154,
 'игрушка': 156,
 'картинка': 158,
 'журнальная;': 159,
 'вдруг': 1071,
 'подлетает': 161,
 'кавалер:': 162,
 '«удостойте': 163,
 'счастия,': 164,
 'сударыня!»': 165,
 'ну,': 4201,
 'видишь:': 167,
 'если': 254,
 'человек': 4187,
 'понятием': 171,
 'армейской': 173,
 'какой': 3785,
 'возьмешь': 176,
 'да': 4214,
 'и': 4215,
 'прищуришься,': 179,
 'отвечаешь:': 180,
 '«извольте,': 181,
 

In [7]:
index2char

{0: '\ufeffсвои',
 1: 'люди',
 2: '–',
 3: 'сочтемся',
 4: 'александр',
 5: 'николаевич',
 6: 'островский',
 7: '«липочка(сидит',
 8: 'у',
 9: 'окна',
 10: 'с',
 11: 'книгой).',
 12: 'какое',
 13: 'приятное',
 14: 'занятие',
 15: 'эти',
 16: 'танцы!',
 17: 'ведь',
 18: 'уж',
 19: 'как',
 20: 'хорошо!',
 21: 'что',
 22: 'может',
 23: 'быть',
 24: 'восхитительнее?',
 25: 'приедешь',
 26: 'в',
 27: 'собрание',
 28: 'али',
 29: 'к',
 30: 'кому',
 31: 'на',
 32: 'свадьбу,',
 33: 'сидишь,',
 34: 'натурально',
 35: '–',
 36: 'вся',
 37: 'в',
 38: 'цветах,',
 39: 'разодета,',
 40: 'как',
 41: 'игрушка',
 42: 'али',
 43: 'картинка',
 44: 'журнальная;',
 45: 'вдруг',
 46: 'подлетает',
 47: 'кавалер:',
 48: '«удостойте',
 49: 'счастия,',
 50: 'сударыня!»',
 51: 'ну,',
 52: 'видишь:',
 53: 'если',
 54: 'человек',
 55: 'с',
 56: 'понятием',
 57: 'али',
 58: 'армейской',
 59: 'какой',
 60: '–',
 61: 'возьмешь',
 62: 'да',
 63: 'и',
 64: 'прищуришься,',
 65: 'отвечаешь:',
 66: '«извольте,',
 67: 'с',

In [8]:
print("Creating input and label text...")
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(chars) - SEQLEN, STEP):
    input_chars.append(chars[i:i + SEQLEN])
    label_chars.append(chars[i + SEQLEN])

Creating input and label text...


In [9]:
input_chars

[['\ufeffсвои',
  'люди',
  '–',
  'сочтемся',
  'александр',
  'николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна'],
 ['люди',
  '–',
  'сочтемся',
  'александр',
  'николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с'],
 ['–',
  'сочтемся',
  'александр',
  'николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).'],
 ['сочтемся',
  'александр',
  'николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).',
  'какое'],
 ['александр',
  'николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).',
  'какое',
  'приятное'],
 ['николаевич',
  'островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).',
  'какое',
  'приятное',
  'занятие'],
 ['островский',
  '«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).',
  'какое',
  'приятное',
  'занятие',
  'эти'],
 ['«липочка(сидит',
  'у',
  'окна',
  'с',
  'книгой).',
  'какое',
  'приятное',
  'занятие',
  'эти',


In [10]:
len(input_chars)

4274

In [11]:
print (nb_chars)

4284


In [12]:
print("Vectorizing input and label text...")
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

Vectorizing input and label text...


In [13]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 40
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [ ]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # testing model
    # randomly choose a row from input_chars, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        ypred = "".join(ypred)
        lst = []
        lst.append(ypred)
        print(lst, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + lst
    print()

Iteration #: 0
Epoch 1/1
4274/4274 [==============================] - 20s 5ms/step - loss: 7.4992
Generating from seed: ['нарядах', 'все.', '(целуясь', 'с', 'липочкой.)', 'вот', 'и', 'до', 'тебя', 'очередь']
['нарядах', 'все.', '(целуясь', 'с', 'липочкой.)', 'вот', 'и', 'до', 'тебя', 'очередь']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']['и']
Iteration #: 1
Epoch 1/1
4274/4274 [==============================] - 32s 8ms/step - loss: 6.8300
Generating from seed: ['по', 'морю,', 'так', 'вот', 'и', 'рябит', 'меланхолия', 'в', 'глазах.', 'устинья']

4274/4274 [==============================] - 14s 3ms/step - loss: 5.4550
Generating from seed: ['а', 'вы', 'купите', 'браслеты', 'с', 'изумрудами.', 'аграфена', 'кондратьевна.', 'куплю,', 'куплю,']
['а', 'вы', 'купите', 'браслеты', 'с', 'изумрудами.', 'аграфена', 'кондратьевна.', 'куплю,', 'куплю,']['что']['не']['за']['самсон']['уходят.']['липочка.']['и']['я']['самсон']['и']['и']['а']['и']['устинья']['не']['кондратьевна.']['за']['не']['и']['самсон']['не']['не']['и']['и']['–']['вот']['и']['фоминишна.']['не']['и']['а']['что']['не']['за']['как']['не']['не']['что']['что']['самсон']['и']['быть']['липочка.']['и']['и']['те']['как']['не']['не']['и']['силыч,']['что']['кондратьевна.']['силыч,']['а']['что']['и']['а']['что']['за']['что']['не']['кондратьевна.']['что']['не']['не']['самсон']['силыч,']['а']['и']['быть']['как']['большов.']['и']['такой']['а']['и']['ты']['не']['силыч,']['а']['что']['что']['не']['и']['не']['не']['что']['что']['как']['самсон']['не']['и']['а']['и']['те']['али']['не']['явлен

['официанты', 'в', 'белых', 'перчатках;', 'я,', 'натурально,', 'в', 'тюлевом', 'либо', 'в']['потом']['какой']['как']['большов.']['еще']['липочка.']['–']['аграфена']['как']['а']['и']['какой']['наумовна.']['как']['али']['вас']['да']['вот']['аграфена']['с']['и']['два…']['два…']['аграфена']['кондратьевна.']['на']['самсон']['уж']['вот']['я']['ты']['как']['как']['вот']['рисположенский.']['явление']['али']['и']['уж']['как']['так']['те']['аграфена']['и']['еще']['отвечаешь:']['и']['за']['такой']['те']['рисположенский.']['что']['вот']['из']['али']['на']['за']['фоминишна.']['устинья']['что']['тебя']['на']['и']['какой']['и']['да']['кондратьевна.']['на']['и']['быть']['а']['явление']['и']['два…']['за']['сысой']['рисположенский.']['и']['что,']['с']['за']['известно,']['ты']['как']['аграфена']['кондратьевна.']['тебя']['силыч,']['липочка.']['я']['ты,']['сказать,']['так']['а']['вот']['рисположенский.']['не']['наумовна.']['два…']['что']
Iteration #: 17
Epoch 1/1
4274/4274 [==============================] 

4274/4274 [==============================] - 16s 4ms/step - loss: 3.4824
Generating from seed: ['это', 'такое!', 'страм', 'встречаться', 'с', 'знакомыми;', 'в', 'целой', 'москве', 'не']
['это', 'такое!', 'страм', 'встречаться', 'с', 'знакомыми;', 'в', 'целой', 'москве', 'не']['могли']['али']['или']['–']['сказать,']['–']['да']['еще']['ах,']['и']['быть']['и']['и']['я']['же']['что']['и']['фоминишна.']['у']['что,']['не']['матушка']['это']['устинья']['это']['не']['ж']['у']['вот,']['липочка.']['что']['отца']['да']['те']['липочка.']['да']['уж']['вот']['на']['вот']['может']['сами!']['а']['что']['вас']['то']['беспременно']['рисположенский.']['армейской']['рисположенский.']['я']['ли?']['рисположенский.']['кому']['на']['как']['еще']['с']['удовольствием!»']['уж']['вот']['вас']['два…']['как']['явление']['старости']['али']['(вздыхает.)']['под']['рисположенский.']['еще']['на']['и']['или']['три…']['и']['рисположенский.']['с']['силыч,']['сударыня']['али']['и']['большов.']['и']['на']['что,']['у']['кондр

4274/4274 [==============================] - 21s 5ms/step - loss: 2.8443
Generating from seed: ['это', 'так:', 'псович,', 'серебряный?', 'по-каковски', 'же', 'это?', 'рисположенский.', 'не', 'умею']
['это', 'так:', 'псович,', 'серебряный?', 'по-каковски', 'же', 'это?', 'рисположенский.', 'не', 'умею']['у']['за']['была']['отца']['звали']['разве']['–']['два…']['бог']['–']['уж']['не']['не']['для']['устинья']['не']['что']['тебя']['чтоб']['что']['и']['я']['я']['и']['и']['так,']['ли']['все']['что']['не']['ж']['как']['как']['ж']['ж']['у']['что']['меня']['не']['я']['вот']['мне']['да']['вот']['уж']['тебе']['не']['я']['что']['как']['как']['за']['и']['в']['да']['как']['что']['как']['ли']['мне']['мне']['и']['что,']['что,']['и']['мне']['не']['мне']['у']['фоминишна.']['как']['тебя']['липочка.']['с']['что,']['как']['чего']['только']['рисположенский.']['это']['аграфена']['аграфена']['бы']['только']['явление']['сысой']['фоминишна.']['ж']['чай,']['да']['это']['устинья']['что']['ж']['а']['кондратьевна.']